In [1]:
import os


In [2]:
%pwd

'c:\\Users\\adkhare\\OneDrive - Informatica\\Desktop\\my_personal\\chicken_disease_MLOps\\chicken_disease_mlops\\research'

In [3]:
os.chdir('../')

In [10]:
from dataclasses import dataclass # dataclass decorator is used to create a class with attributes and methods
from pathlib import Path

@dataclass(frozen=True) # frozen=True makes the class immutable
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )    
        
        return data_ingestion_config
    

In [13]:
import os
import zipfile
import urllib.request as request
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
                )
            logger.info(f"{filename} downloaded successfully with following info: {headers}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists of size {get_size(Path(self.config.local_data_file))}")  
            
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        extracts the zip file into the data directory
        function returns None
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e    

[2023-07-03 14:27:12,104: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-03 14:27:12,107: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-03 14:27:12,108: INFO: common: created directory at: artifacts]
[2023-07-03 14:27:12,111: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-03 14:27:12,111: INFO: 778408504: File artifacts/data_ingestion/data.zip already exists of size ~ 11345 KB]
